# 🏋️‍♂️ Personalized Multi-Agent RAG with Heart Rate Analysis using Azure AI Foundry 🥑❤️

Welcome to this advanced workshop where we'll build a personalized multi-agent Retrieval-Augmented Generation (RAG) pipeline using AutoGen 0.4.7 with Azure AI Foundry. Our team of agents will collaborate to provide fitness and health recommendations based on heart rate data analysis!

## 0. Install Required Packages

Let's first install all the required packages for this notebook. This may take a few minutes.

In [ ]:
# Install required packages
import sys
import subprocess
import importlib.util

# Define required packages
packages = [
    "autogen==0.4.7",  # This should install autogen_* packages
    "autogen-agentchat==0.4.7",
    "autogen-core==0.4.7",
    "autogen-ext==0.4.7",
    "pandas",
    "numpy",
    "matplotlib",
    "azure-core",
    "python-dotenv"  # For managing environment variables
]

def check_package(package_name):
    """Check if package is installed"""
    package_name = package_name.split('==')[0]  # Remove version specifier if present
    return importlib.util.find_spec(package_name) is not None

# Install any missing packages
missing_packages = [pkg for pkg in packages if not check_package(pkg.split('==')[0])]
if missing_packages:
    print(f"Installing missing packages: {', '.join(missing_packages)}")
    for package in missing_packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✅ Successfully installed {package}")
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install {package}: {e}")
else:
    print("All required packages are already installed.")

# Alternative method using pip directly (uncomment if needed)
# !pip install autogen==0.4.7 autogen-agentchat==0.4.7 autogen-core==0.4.7 autogen-ext==0.4.7 pandas numpy matplotlib azure-core python-dotenv

print("\n✅ Package installation complete!")

## 1. Setup

Let's import the necessary libraries and set up our model client using Azure AI Foundry. Make sure your environment variable `GITHUB_TOKEN` is set with your personal access token.

In [ ]:
import os
import asyncio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Import AutoGen agents and required modules
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

# Import the Azure AI Foundry model client from AutoGen extensions
from autogen_ext.models.azure import AzureAIChatCompletionClient
from autogen_core.models import UserMessage
from azure.core.credentials import AzureKeyCredential

# Create the model client using Azure AI Foundry
try:
    model_client = AzureAIChatCompletionClient(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        endpoint="https://models.inference.ai.azure.com",
        credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
        model_info={
            "json_output": False,
            "function_calling": False,
            "vision": False,
            "family": "unknown"
        }
    )
    print("✅ Azure AI Foundry model client created successfully!")
except Exception as e:
    print(f"⚠️ Error creating Azure AI Foundry model client: {e}")
    print("⚠️ Using a placeholder model client for demonstration purposes.")
    # Create a simple placeholder for the model client
    class PlaceholderModelClient:
        def generate(self, messages, **kwargs):
            return ["This is a placeholder response. Azure AI Foundry client not configured."]
    model_client = PlaceholderModelClient()

## 2. Create Sample Health Data and Retrieval Tool

We'll define a small list of health tips and a simple retrieval function. This function simulates retrieving relevant health tips based on keywords in the user's query.

In [ ]:
# Define sample health tips
health_tips = [
    {"id": "tip1", "content": "Do a 10-minute HIIT workout to boost your metabolism.", "source": "Fitness Guru"},
    {"id": "tip2", "content": "Take a brisk 15-minute walk to clear your mind and improve circulation.", "source": "Health Coach"},
    {"id": "tip3", "content": "Stretch for 5 minutes every hour if you're sitting at a desk.", "source": "Wellness Expert"},
    {"id": "tip4", "content": "Incorporate strength training twice a week for overall fitness.", "source": "Personal Trainer"},
    {"id": "tip5", "content": "Drink water regularly to stay hydrated during workouts.", "source": "Nutritionist"},
    {"id": "tip6", "content": "For low-intensity recovery days, try yoga or gentle swimming.", "source": "Recovery Specialist"},
    {"id": "tip7", "content": "If your heart rate was elevated yesterday, focus on slow, deep breathing exercises today.", "source": "Breathing Coach"},
    {"id": "tip8", "content": "Morning walks are ideal when your body is naturally in a recovery state.", "source": "Circadian Expert"},
    {"id": "tip9", "content": "Adjust your hydration based on your previous day's heart rate - higher rates mean more water today.", "source": "Hydration Specialist"},
    {"id": "tip10", "content": "Schedule high-intensity workouts during your body's natural energy peaks based on heart rate patterns.", "source": "Performance Coach"}
]

def retrieve_tips(query: str) -> str:
    """Return health tips whose content contains keywords from the query."""
    query_lower = query.lower()
    relevant = []
    for tip in health_tips:
        # Check if any word in the query is in the tip content
        if any(word in tip["content"].lower() for word in query_lower.split()):
            relevant.append(f"Source: {tip['source']} => {tip['content']}")
    if not relevant:
        # If no tips match, return all tips (for demo purposes)
        relevant = [f"Source: {tip['source']} => {tip['content']}" for tip in health_tips[:5]]
    return "\n".join(relevant)

print("✅ Sample health tips and retrieval tool created!")

## 3. Generate or Import Heart Rate Data

You can choose to generate synthetic heart rate data for testing purposes or import real data from a CSV file.

In [ ]:
# Set a seed for reproducibility
np.random.seed(42)

# Option to use synthetic data or load from CSV
use_synthetic_data = False  # Set to False to load from CSV file

if use_synthetic_data:
    # Create date range for yesterday (24 hours with hourly data)
    end_time = datetime.now().replace(minute=0, second=0, microsecond=0)
    start_time = end_time - timedelta(days=1)
    date_range = pd.date_range(start=start_time, end=end_time, freq='H')

    # Create baseline heart rate pattern (higher during day, lower at night)
    hours = np.array([(t.hour + 1) for t in date_range])
    # Heart rate is lower at night (hours 0-6), rises during day, peaks in afternoon
    baseline_hr = 50 + 30 * np.sin(np.pi * (hours - 6) / 12) ** 2

    # Add some random variation
    heart_rates = baseline_hr + np.random.normal(0, 5, size=len(date_range))
    heart_rates = np.clip(heart_rates, 50, 120).astype(int)  # Clip to reasonable heart rate range

    # Create a DataFrame with the synthetic data
    df_synthetic = pd.DataFrame({
        'hour': date_range,
        'hr_mean': heart_rates,
        'hr_std': np.random.uniform(2, 8, size=len(date_range)),
        'hr_min': [max(hr - np.random.randint(5, 15), 45) for hr in heart_rates],
        'hr_max': [min(hr + np.random.randint(5, 20), 130) for hr in heart_rates],
        'flow_intensity': np.random.uniform(30, 70, size=len(date_range)),
        'likelihood_calm': np.random.beta(2, 2, size=len(date_range)),
        'likelihood_excited': np.random.beta(1.5, 3, size=len(date_range)),
        'likelihood_frustrated': np.random.beta(1, 4, size=len(date_range))
    })

    # Adjust the calm likelihood to be higher when heart rate is lower
    df_synthetic['likelihood_calm'] = 1 - (df_synthetic['hr_mean'] - df_synthetic['hr_mean'].min()) / (df_synthetic['hr_mean'].max() - df_synthetic['hr_mean'].min())
    df_synthetic['likelihood_calm'] = 0.3 + 0.6 * df_synthetic['likelihood_calm']  # Scale to 0.3-0.9

    # Adjust excited likelihood to be higher when heart rate is higher
    df_synthetic['likelihood_excited'] = (df_synthetic['hr_mean'] - df_synthetic['hr_mean'].min()) / (df_synthetic['hr_mean'].max() - df_synthetic['hr_mean'].min())
    df_synthetic['likelihood_excited'] = 0.2 + 0.7 * df_synthetic['likelihood_excited']  # Scale to 0.2-0.9

    # Display the synthetic data
    print("Generated synthetic heart rate data:")
    display(df_synthetic.head())

    # Plot the synthetic heart rate data
    plt.figure(figsize=(12, 6))
    plt.plot(df_synthetic['hour'], df_synthetic['hr_mean'], label='Mean HR')
    plt.fill_between(df_synthetic['hour'], 
                     df_synthetic['hr_mean'] - df_synthetic['hr_std'],
                     df_synthetic['hr_mean'] + df_synthetic['hr_std'],
                     alpha=0.2)
    plt.title('Synthetic Heart Rate Over Time')
    plt.xlabel('Time')
    plt.ylabel('Heart Rate (BPM)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

    # Use this synthetic data as our hourly_data for analysis
    hourly_data = df_synthetic.copy()
    
    print("✅ Synthetic heart rate data generated and visualized!")

else:
    # Load data from CSV file
    try:
        csv_file = "example-data/20250417_health_data.csv"
        print(f"Loading heart rate data from {csv_file}...")
        
        # Read the CSV file
        df_real = pd.read_csv(csv_file)
        
        # Convert timestamp column to datetime if it exists
        if 'timestamp' in df_real.columns:
            timezone_offset = 60*60*5
            df_real['hour'] = pd.to_datetime(df_real['timestamp']-timezone_offset, unit='s')
        elif 'date' in df_real.columns:
            df_real['hour'] = pd.to_datetime(df_real['date'])
        else:
            # If no timestamp column, create one from current time
            end_time = datetime.now().replace(minute=0, second=0, microsecond=0)
            start_time = end_time - timedelta(days=1)
            df_real['hour'] = pd.date_range(start=start_time, end=end_time, freq='H')[:len(df_real)]
        
        # Ensure all required columns exist
        if 'heart_rate' in df_real.columns:
            df_real['hr_mean'] = df_real['heart_rate']
        else:
            print("⚠️ No heart rate column found in CSV. Using random values.")
            df_real['hr_mean'] = np.random.randint(60, 100, size=len(df_real))
            
        # Calculate or generate other required columns if they don't exist
        if 'hr_std' not in df_real.columns:
            df_real['hr_std'] = df_real['hr_mean'] * 0.1  # Estimate std as 10% of mean
            
        if 'hr_min' not in df_real.columns:
            df_real['hr_min'] = df_real['hr_mean'] - df_real['hr_std']
            
        if 'hr_max' not in df_real.columns:
            df_real['hr_max'] = df_real['hr_mean'] + df_real['hr_std']
            
        # Generate mood likelihood columns if they don't exist
        for col in ['likelihood_calm', 'likelihood_excited', 'likelihood_frustrated']:
            if col not in df_real.columns:
                if col == 'likelihood_calm':
                    # Calm is higher when heart rate is lower
                    normalized_hr = (df_real['hr_mean'] - df_real['hr_mean'].min()) / (df_real['hr_mean'].max() - df_real['hr_mean'].min())
                    df_real[col] = 0.3 + 0.6 * (1 - normalized_hr)
                elif col == 'likelihood_excited':
                    # Excited is higher when heart rate is higher
                    normalized_hr = (df_real['hr_mean'] - df_real['hr_mean'].min()) / (df_real['hr_mean'].max() - df_real['hr_mean'].min())
                    df_real[col] = 0.2 + 0.7 * normalized_hr
                else:
                    # Frustrated is random for this example
                    df_real[col] = np.random.beta(1, 4, size=len(df_real))
        
        if 'flow_intensity' not in df_real.columns:
            df_real['flow_intensity'] = np.random.uniform(30, 70, size=len(df_real))
            
        # Display the loaded data
        print("Loaded heart rate data from CSV:")
        display(df_real.head())
        
        # Plot the heart rate data
        plt.figure(figsize=(12, 6))
        plt.plot(df_real['hour'], df_real['hr_mean'], label='Mean HR')
        if 'hr_std' in df_real.columns:
            plt.fill_between(df_real['hour'], 
                           df_real['hr_mean'] - df_real['hr_std'],
                           df_real['hr_mean'] + df_real['hr_std'],
                           alpha=0.2)
        plt.title('Heart Rate Over Time from CSV Data')
        plt.xlabel('Time')
        plt.ylabel('Heart Rate (BPM)')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.show()
        
        # Use this loaded data as our hourly_data for analysis
        hourly_data = df_real.copy()
        
        print("✅ Heart rate data loaded from CSV and visualized!")
        
    except Exception as e:
        print(f"❌ Error loading CSV data: {e}")
        print("Falling back to synthetic data...")
        
        # Fall back to synthetic data generation
        end_time = datetime.now().replace(minute=0, second=0, microsecond=0)
        start_time = end_time - timedelta(days=1)
        date_range = pd.date_range(start=start_time, end=end_time, freq='H')
        
        hours = np.array([(t.hour + 1) for t in date_range])
        baseline_hr = 50 + 30 * np.sin(np.pi * (hours - 6) / 12) ** 2
        heart_rates = baseline_hr + np.random.normal(0, 5, size=len(date_range))
        heart_rates = np.clip(heart_rates, 50, 120).astype(int)
        
        # Create a DataFrame with synthetic data as fallback
        df_synthetic = pd.DataFrame({
            'hour': date_range,
            'hr_mean': heart_rates,
            'hr_std': np.random.uniform(2, 8, size=len(date_range)),
            'hr_min': [max(hr - np.random.randint(5, 15), 45) for hr in heart_rates],
            'hr_max': [min(hr + np.random.randint(5, 20), 130) for hr in heart_rates],
            'flow_intensity': np.random.uniform(30, 70, size=len(date_range)),
            'likelihood_calm': np.random.beta(2, 2, size=len(date_range)),
            'likelihood_excited': np.random.beta(1.5, 3, size=len(date_range)),
            'likelihood_frustrated': np.random.beta(1, 4, size=len(date_range))
        })
        
        # Adjust the calm and excited likelihoods
        df_synthetic['likelihood_calm'] = 1 - (df_synthetic['hr_mean'] - df_synthetic['hr_mean'].min()) / (df_synthetic['hr_mean'].max() - df_synthetic['hr_mean'].min())
        df_synthetic['likelihood_calm'] = 0.3 + 0.6 * df_synthetic['likelihood_calm']
        
        df_synthetic['likelihood_excited'] = (df_synthetic['hr_mean'] - df_synthetic['hr_mean'].min()) / (df_synthetic['hr_mean'].max() - df_synthetic['hr_mean'].min())
        df_synthetic['likelihood_excited'] = 0.2 + 0.7 * df_synthetic['likelihood_excited']
        
        hourly_data = df_synthetic.copy()
        
        print("✅ Fallback to synthetic heart rate data complete!")

## 4. Analyzing Heart Rate Volatility and Cyclical Patterns

Now we'll analyze the heart rate data in three ways to understand heart strain and identify patterns:

1. Volatility analysis to understand heart strain
2. Cyclical pattern analysis to understand daily rhythms
3. Recovery period identification to find optimal rest times

In [ ]:
# 1. HEART RATE VOLATILITY ANALYSIS
# Calculate rolling volatility metrics with a 3-hour window (smaller for our synthetic data)
window_size = 3  # 3-hour window
hourly_data['hr_volatility'] = hourly_data['hr_std'].rolling(window=window_size, min_periods=1).mean()
hourly_data['hr_range'] = (hourly_data['hr_max'] - hourly_data['hr_min']).rolling(window=window_size, min_periods=1).mean()

# Create heart strain index based on volatility and range
hourly_data['strain_index'] = (hourly_data['hr_volatility'] * 0.5 + 
                              hourly_data['hr_range'] * 0.3 + 
                              hourly_data['hr_mean'] * 0.2) / 10

# Plot heart strain index
plt.figure(figsize=(12, 6))
plt.plot(hourly_data['hour'], hourly_data['strain_index'], 'r-', label='Heart Strain Index')
plt.title('Heart Strain Index Over Time (Higher = More Strain)')
plt.xlabel('Time')
plt.ylabel('Strain Index')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

# 2. CYCLICAL PATTERN ANALYSIS
# Add time features for cyclical analysis
hourly_data['hour_of_day'] = hourly_data['hour'].dt.hour
hourly_data['day_of_week'] = hourly_data['hour'].dt.dayofweek

# Calculate average heart rate by hour of day
hr_by_hour = hourly_data.groupby('hour_of_day')['hr_mean'].mean().reset_index()

plt.figure(figsize=(12, 6))
plt.bar(hr_by_hour['hour_of_day'], hr_by_hour['hr_mean'], color='skyblue')
plt.title('Average Heart Rate by Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Average Heart Rate (BPM)')
plt.xticks(range(0, 24, 2))
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.show()

# 3. RECOVERY PERIODS IDENTIFICATION
# Identify periods of low heart rate (recovery periods)
recovery_threshold = hourly_data['hr_mean'].quantile(0.25)  # Bottom 25% as recovery periods
hourly_data['is_recovery'] = hourly_data['hr_mean'] <= recovery_threshold

# Identify optimal recovery periods (low HR, low volatility, high calmness)
hourly_data['optimal_recovery'] = ((hourly_data['hr_mean'] <= recovery_threshold) & 
                                 (hourly_data['hr_volatility'] <= hourly_data['hr_volatility'].quantile(0.3)) &
                                 (hourly_data['likelihood_calm'] >= hourly_data['likelihood_calm'].quantile(0.7)))

# Show recovery periods
recovery_hours = hourly_data[hourly_data['optimal_recovery'] == True][['hour', 'hr_mean', 'hr_volatility', 'likelihood_calm']]
print("\nOptimal Recovery Periods Identified:")
display(recovery_hours)

# Visualize recovery periods
plt.figure(figsize=(12, 6))
plt.scatter(hourly_data['hour'], hourly_data['hr_mean'], 
           c=hourly_data['optimal_recovery'].map({True: 'green', False: 'gray'}),
           s=100, alpha=0.7)
plt.title('Heart Rate with Optimal Recovery Periods Highlighted')
plt.xlabel('Time')
plt.ylabel('Heart Rate (BPM)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

# Generate heart rate analysis summary
print("\nHeart Rate Analysis Summary:")
print(f"Average Heart Rate: {hourly_data['hr_mean'].mean():.1f} BPM")
print(f"Average Heart Rate Volatility: {hourly_data['hr_volatility'].mean():.2f}")
print(f"Peak Heart Strain Period: {hourly_data.loc[hourly_data['strain_index'].idxmax(), 'hour']}")
print(f"Optimal Recovery Hours: {', '.join([str(h.hour) + ':00' for h in recovery_hours['hour']])}")

print("✅ Heart rate analysis complete!")

## 4.1. Analyzing Heart Rate Phase Patterns

Now we'll identify in-phase and out-of-phase periods based on heart rate cyclical patterns. This can help identify when a person's heart rate is aligned with their expected daily rhythm versus when it deviates from the pattern.

In [ ]:
# HEART RATE PHASE PATTERN ANALYSIS
# Calculate the expected heart rate pattern based on time of day
# First, create a model of expected heart rate by hour
expected_hr_by_hour = hr_by_hour.copy()  # Using the hourly averages we calculated earlier

# Create a lookup dictionary for expected HR by hour
expected_hr_dict = dict(zip(expected_hr_by_hour['hour_of_day'], expected_hr_by_hour['hr_mean']))

# Calculate the expected heart rate for each time point
hourly_data['expected_hr'] = hourly_data['hour_of_day'].map(expected_hr_dict)

# Calculate the deviation from expected pattern
hourly_data['hr_deviation'] = hourly_data['hr_mean'] - hourly_data['expected_hr']

# Define phase status
# In-phase: deviation is within 10% of expected HR
# Out-of-phase: deviation is greater than 10% of expected HR
threshold_percent = 0.10
hourly_data['threshold'] = hourly_data['expected_hr'] * threshold_percent
hourly_data['phase_status'] = 'in-phase'
hourly_data.loc[hourly_data['hr_deviation'].abs() > hourly_data['threshold'], 'phase_status'] = 'out-of-phase'

# Calculate percentage of time in each phase
phase_counts = hourly_data['phase_status'].value_counts(normalize=True) * 100
print("Percentage of time in each phase:")
for phase, percentage in phase_counts.items():
    print(f"{phase}: {percentage:.1f}%")

# Identify longest continuous out-of-phase period
hourly_data['phase_change'] = hourly_data['phase_status'].ne(hourly_data['phase_status'].shift()).cumsum()
phase_groups = hourly_data.groupby(['phase_status', 'phase_change'])

out_of_phase_periods = []
for (status, _), group in phase_groups:
    if status == 'out-of-phase' and len(group) >= 2:  # At least 2 hours
        start_time = group['hour'].min()
        end_time = group['hour'].max()
        duration = len(group)
        out_of_phase_periods.append((start_time, end_time, duration))

if out_of_phase_periods:
    # Sort by duration (longest first)
    out_of_phase_periods.sort(key=lambda x: x[2], reverse=True)
    longest_period = out_of_phase_periods[0]
    print(f"\nLongest out-of-phase period: {longest_period[2]}")
    print(f"From {longest_period[0]} to {longest_period[1]}")
else:
    print("\nNo continuous out-of-phase periods found.")

# Visualize phases and deviations
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Plot 1: Actual vs Expected HR with phase coloring
colors = {'in-phase': 'green', 'out-of-phase': 'red'}
for phase in ['in-phase', 'out-of-phase']:
    phase_data = hourly_data[hourly_data['phase_status'] == phase]
    ax1.scatter(phase_data['hour'], phase_data['hr_mean'], 
               color=colors[phase], label=phase, alpha=0.7, s=50)

ax1.plot(hourly_data['hour'], hourly_data['expected_hr'], 'b--', label='Expected HR Pattern', linewidth=2)
ax1.set_title('Heart Rate: Actual vs Expected Pattern with Phase Status')
ax1.set_ylabel('Heart Rate (BPM)')
ax1.legend()
ax1.grid(True, linestyle='--', alpha=0.5)

# Plot 2: HR Deviation with threshold bands
ax2.axhline(y=0, color='k', linestyle='-', alpha=0.3)
ax2.fill_between(hourly_data['hour'], 
                -hourly_data['threshold'], 
                hourly_data['threshold'], 
                color='green', alpha=0.2, label='In-phase threshold')

# Plot deviation line colored by phase status
for phase in ['in-phase', 'out-of-phase']:
    phase_data = hourly_data[hourly_data['phase_status'] == phase]
    ax2.plot(phase_data['hour'], phase_data['hr_deviation'], 
            color=colors[phase], marker='o', linestyle='-', alpha=0.7)

ax2.set_title('Heart Rate Deviation from Expected Pattern')
ax2.set_xlabel('Time')
ax2.set_ylabel('Deviation (BPM)')
ax2.legend()
ax2.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# Update our personalization function to include phase information
hourly_data['phase_status'] = hourly_data['phase_status']

print("\n✅ Heart rate phase analysis complete!")

## 4.2. Real-time Heart Strain Likelihood Indicator

Now let's create a function that estimates heart strain likelihood based on a user's message sentiment and their most recent heart rate data. This can provide real-time guidance on potential heart strain risks.

In [ ]:
def analyze_message_sentiment(message):
    """Simple sentiment analysis for user messages to detect potential stress indicators."""
    # List of words that might indicate stress or anxiety
    stress_words = ['stress', 'stressed', 'anxious', 'worried', 'tired', 'exhausted', 
                   'overwhelmed', 'panic', 'afraid', 'fear', 'tension', 'pressure',
                   'difficult', 'hard', 'problem', 'trouble', 'issue', 'concerned']
    
    # Count stress indicators in message
    message_lower = message.lower()
    stress_indicators = sum(word in message_lower for word in stress_words)
    
    # Normalize to a 0-1 scale, with some dampening to avoid extremes
    sentiment_score = min(0.8, stress_indicators * 0.2) if stress_indicators > 0 else 0
    
    # Look for explicit mentions of feeling good/relaxed
    positive_words = ['relaxed', 'calm', 'happy', 'great', 'good', 'fine', 'well', 'rested']
    positive_indicators = sum(word in message_lower for word in positive_words)
    
    # Reduce score if positive indicators are present
    if positive_indicators > 0:
        sentiment_score = max(0, sentiment_score - 0.3 * positive_indicators)
    
    # Return sentiment score where higher is more stressed
    return sentiment_score

def get_latest_hr_data(minutes=5):
    """Get the latest heart rate data for the specified number of minutes.
    
    For the purpose of this demo, we'll randomly select data points from our existing dataset
    to simulate real-time data collection from a wearable device.
    """
    # In a real implementation, this would fetch actual real-time data from a wearable device API
    # For demonstration, we'll sample from our existing data
    sample_size = min(minutes, len(hourly_data))
    latest_data = hourly_data.sample(sample_size).copy()
    
    # Add some randomness to simulate recent fluctuations
    latest_data['hr_mean'] = latest_data['hr_mean'] + np.random.normal(0, 3, size=len(latest_data))
    latest_data['hr_volatility'] = latest_data['hr_volatility'] * (1 + np.random.uniform(-0.1, 0.1, size=len(latest_data)))
    
    return latest_data

def estimate_heart_strain_likelihood(message="", recent_minutes=5):
    """Estimate heart strain likelihood based on recent HR data and user message."""
    # Get the latest heart rate data
    latest_data = get_latest_hr_data(minutes=recent_minutes)
    
    # Extract heart rate metrics
    current_hr = latest_data['hr_mean'].mean()
    current_volatility = latest_data['hr_volatility'].mean()
    is_out_of_phase = (latest_data['phase_status'] == 'out-of-phase').mean() > 0.5
    
    # Analyze the message sentiment (0-1 scale, higher means more stressed)
    sentiment_score = analyze_message_sentiment(message) if message else 0.0
    
    # Calculate heart strain factors
    hr_factor = (current_hr - hourly_data['hr_mean'].min()) / (hourly_data['hr_mean'].max() - hourly_data['hr_mean'].min())
    volatility_factor = (current_volatility - hourly_data['hr_volatility'].min()) / (hourly_data['hr_volatility'].max() - hourly_data['hr_volatility'].min())
    phase_factor = 0.2 if is_out_of_phase else 0.0
    
    # Combine factors (weighted average)
    strain_likelihood = (0.4 * hr_factor + 
                        0.3 * volatility_factor + 
                        0.2 * sentiment_score + 
                        0.1 * phase_factor)
    
    # Scale to 0-100%
    strain_percent = min(100, max(0, round(strain_likelihood * 100)))
    
    # Determine strain level category
    if strain_percent < 30:
        level = "Low"
        color = "green"
    elif strain_percent < 60:
        level = "Moderate"
        color = "orange"
    else:
        level = "High"
        color = "red"
    
    return {
        "strain_likelihood": strain_percent,
        "strain_level": level,
        "color": color,
        "current_hr": current_hr,
        "current_volatility": current_volatility,
        "is_out_of_phase": is_out_of_phase,
        "message_sentiment": sentiment_score
    }

def display_strain_indicator(result):
    """Display a visual indicator of heart strain likelihood."""
    # Create a visualization of the strain likelihood
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4), gridspec_kw={'width_ratios': [1, 2]})
    
    # Gauge-style visualization
    strain = result["strain_likelihood"]
    color = result["color"]
    
    # Create a half-circle gauge
    theta = np.linspace(0, np.pi, 100)
    r = 1.0
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    
    # Draw the gauge background
    ax1.plot(x, y, 'k-', linewidth=2)
    ax1.fill_between(x, 0, y, color='lightgray', alpha=0.3)
    
    # Calculate the angle for the needle based on strain
    needle_angle = np.pi * strain / 100
    needle_x = [0, r * np.cos(needle_angle)]
    needle_y = [0, r * np.sin(needle_angle)]
    
    # Draw the needle
    ax1.plot(needle_x, needle_y, color=color, linewidth=3)
    
    # Add labels
    ax1.text(-1.1, -0.15, "Low", fontsize=12, ha='left')
    ax1.text(0, -0.15, "Moderate", fontsize=12, ha='center')
    ax1.text(1.1, -0.15, "High", fontsize=12, ha='right')
    ax1.text(0, 0.5, f"{strain}%", fontsize=18, ha='center', weight='bold', color=color)
    ax1.text(0, 0.3, f"{result['strain_level']} Strain Likelihood", fontsize=14, ha='center')
    
    # Adjust the aspect ratio and remove axes
    ax1.set_aspect('equal')
    ax1.axis('off')
    
    # Plot the contributing factors as a horizontal bar chart
    factors = {
        'Heart Rate': result['current_hr'],
        'HR Volatility': result['current_volatility'],
        'Message Sentiment': result['message_sentiment'] * 10,  # Scale up for visibility
        'Phase Status': 8 if result['is_out_of_phase'] else 2  # Binary factor
    }
    
    y_pos = np.arange(len(factors))
    factor_values = list(factors.values())
    
    ax2.barh(y_pos, factor_values, color=['skyblue', 'lightgreen', 'plum', 'khaki'])
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(factors.keys())
    ax2.set_title('Contributing Factors')
    ax2.grid(True, linestyle='--', alpha=0.5, axis='x')
    
    plt.tight_layout()
    plt.show()
    
    # Print recommendations based on strain level
    print(f"\nHeart Strain Likelihood: {strain}% ({result['strain_level']})")
    print(f"Current Heart Rate: {result['current_hr']:.1f} BPM")
    
    if result['strain_level'] == "High":
        print("\nRecommendation: Consider taking a break. Practice deep breathing and hydrate.")
    elif result['strain_level'] == "Moderate":
        print("\nRecommendation: Monitor your heart rate. Take a short break if you can.")
    else:
        print("\nRecommendation: Your heart strain is low. Continue your activities as normal.")

# Interactive input for testing the heart strain indicator
def test_heart_strain_indicator():
    user_message = input("How are you feeling right now? ")
    result = estimate_heart_strain_likelihood(message=user_message, recent_minutes=5)
    display_strain_indicator(result)
    return result

# Test with a sample message
sample_result = estimate_heart_strain_likelihood(message="I'm feeling a bit stressed about this project deadline", recent_minutes=5)
display_strain_indicator(sample_result)

# Uncomment the line below to test with your own input
# test_heart_strain_indicator()

print("\n✅ Heart strain indicator created!")

## 4.3. Personalized Nutrition Recommendations Based on Heart Rate Data

Let's create a system that provides personalized food recommendations based on the user's heart rate data, recent meals, and desired mood states. This integrates with nutrition data that could be pulled from apps like Alma.

In [ ]:
# Simulated nutrition database for demonstration purposes
# In a real implementation, this would connect to an API like Alma's
# reference a real nutrition database or use a tool such as https://www.alma.food/
nutrition_database = {
    "excited": {
        "foods": ["dark chocolate", "banana", "coffee", "green tea", "berries", "nuts", "oatmeal with fruit", 
                 "greek yogurt with honey", "smoothie with spinach and berries", "eggs with avocado", 
                 "quinoa bowl", "chia pudding", "kombucha"],
        "nutrients": ["vitamin B complex", "magnesium", "caffeine (moderate)", "complex carbohydrates", "protein"],
        "avoid": ["heavy, fatty meals", "simple sugars", "excessive caffeine", "processed foods"]
    },
    "calm": {
        "foods": ["chamomile tea", "warm milk", "turkey", "bananas", "oatmeal", "almonds", "fatty fish", 
                 "dark leafy greens", "lentil soup", "sweet potatoes", "whole grain pasta", "kiwi", "cherries"],
        "nutrients": ["tryptophan", "magnesium", "vitamin B6", "omega-3 fatty acids", "melatonin"],
        "avoid": ["caffeine", "alcohol", "spicy foods", "high sugar foods", "processed snacks"]
    },
    "focused": {
        "foods": ["blueberries", "fatty fish", "dark chocolate", "nuts", "avocados", "leafy greens", 
                 "eggs", "broccoli", "pumpkin seeds", "green tea", "coffee", "water", "beet juice"],
        "nutrients": ["omega-3", "antioxidants", "choline", "vitamin E", "polyphenols", "moderate caffeine"],
        "avoid": ["high glycemic carbs", "excessive sugar", "dehydration", "very heavy meals"]
    },
    "energized": {
        "foods": ["oatmeal", "quinoa", "sweet potatoes", "bananas", "apples", "oranges", "greek yogurt", 
                 "lentils", "chickpeas", "lean protein", "beets", "pomegranate", "nut butter", "honey"],
        "nutrients": ["complex carbohydrates", "B vitamins", "iron", "potassium", "protein", "fiber"],
        "avoid": ["simple sugars", "processed foods", "excessive fat", "dehydration"]
    },
    "recovery": {
        "foods": ["salmon", "tart cherry juice", "watermelon", "sweet potatoes", "eggs", "leafy greens", 
                 "turmeric (with black pepper)", "ginger", "berries", "pomegranate", "greek yogurt"],
        "nutrients": ["omega-3 fatty acids", "antioxidants", "protein", "vitamin C", "potassium", "magnesium"],
        "avoid": ["alcohol", "excessive caffeine", "processed foods", "fried foods"]
    }
}

# Sample meal database with calorie and macronutrient information
meal_database = {
    "oatmeal with berries": {"calories": 300, "protein": 10, "carbs": 50, "fat": 6, "mood": ["excited", "energized"]},
    "greek yogurt with honey": {"calories": 250, "protein": 20, "carbs": 30, "fat": 5, "mood": ["excited", "focused"]},
    "salmon with sweet potatoes": {"calories": 450, "protein": 35, "carbs": 40, "fat": 15, "mood": ["recovery", "focused"]},
    "chicken salad": {"calories": 350, "protein": 30, "carbs": 15, "fat": 20, "mood": ["energized", "focused"]},
    "lentil soup": {"calories": 300, "protein": 15, "carbs": 45, "fat": 5, "mood": ["calm", "recovery"]},
    "avocado toast": {"calories": 350, "protein": 12, "carbs": 30, "fat": 20, "mood": ["focused", "energized"]},
    "smoothie with spinach and berries": {"calories": 200, "protein": 5, "carbs": 35, "fat": 2, "mood": ["excited", "energized"]},
    "eggs with avocado": {"calories": 400, "protein": 20, "carbs": 10, "fat": 30, "mood": ["focused", "energized"]},
    "dark chocolate": {"calories": 150, "protein": 2, "carbs": 15, "fat": 10, "mood": ["excited", "focused"]},
    "chamomile tea": {"calories": 0, "protein": 0, "carbs": 0, "fat": 0, "mood": ["calm"]},
    "grilled chicken sandwich": {"calories": 450, "protein": 35, "carbs": 40, "fat": 15, "mood": ["energized", "recovery"]},
    "pasta with tomato sauce": {"calories": 400, "protein": 12, "carbs": 70, "fat": 8, "mood": ["calm", "energized"]},
    "steak with vegetables": {"calories": 550, "protein": 40, "carbs": 20, "fat": 35, "mood": ["recovery", "energized"]},
    "fruit salad": {"calories": 150, "protein": 2, "carbs": 35, "fat": 0, "mood": ["excited", "energized"]},
    "quinoa bowl": {"calories": 450, "protein": 15, "carbs": 65, "fat": 12, "mood": ["excited", "focused", "energized"]}
}

# User's last meal tracking
user_last_meal = {
    "meal": "chicken salad",
    "time": datetime.now() - timedelta(hours=4),  # Assume eaten 4 hours ago
    "calories": 350,
    "protein": 30,
    "carbs": 15,
    "fat": 20
}

# Daily calorie targets (would be personalized in a real app)
user_calorie_targets = {
    "daily_total": 2000,
    "breakfast": 500,
    "lunch": 600,
    "dinner": 600,
    "snacks": 300
}

def get_mood_from_phrase(phrase):
    """Extract the desired mood from a user's query phrase."""
    mood_keywords = {
        "excited": ["excited", "energetic", "upbeat", "lively", "enthusiastic", "pumped", "motivated"],
        "calm": ["calm", "relaxed", "peaceful", "tranquil", "serene", "chill", "soothing", "stress-free"],
        "focused": ["focused", "concentrate", "attentive", "alert", "productive", "sharp", "clarity"],
        "energized": ["energized", "active", "vibrant", "vigor", "stamina", "strength", "power"],
        "recovery": ["recovery", "rest", "restore", "replenish", "rejuvenate", "healing", "recuperate"]
    }
    
    phrase_lower = phrase.lower()
    
    # Check for mood keywords in the phrase
    for mood, keywords in mood_keywords.items():
        if any(keyword in phrase_lower for keyword in keywords):
            return mood
    
    # Default to energized if no mood is detected
    return "energized"

def get_meal_time_from_phrase(phrase):
    """Extract the meal time from a user's query phrase."""
    phrase_lower = phrase.lower()
    
    if "breakfast" in phrase_lower or "morning" in phrase_lower:
        return "breakfast"
    elif "lunch" in phrase_lower or "noon" in phrase_lower or "midday" in phrase_lower:
        return "lunch"
    elif "dinner" in phrase_lower or "evening" in phrase_lower or "night" in phrase_lower:
        return "dinner"
    elif "snack" in phrase_lower:
        return "snack"
    
    # Check for time of day references
    if "afternoon" in phrase_lower:
        current_hour = datetime.now().hour
        return "lunch" if current_hour < 14 else "snack"
    
    # Default to the next upcoming meal based on current time
    current_hour = datetime.now().hour
    if current_hour < 10:
        return "breakfast"
    elif current_hour < 14:
        return "lunch"
    elif current_hour < 18:
        return "snack"
    else:
        return "dinner"

def get_remaining_calories():
    """Calculate remaining calories for the day based on last meal."""
    # This would be a more sophisticated calculation in a real app with full meal history
    # For demo purposes, we'll assume the user has consumed the last meal's calories
    # plus 50% of their daily target (simulating previous meals)
    consumed_so_far = user_last_meal["calories"] + (user_calorie_targets["daily_total"] * 0.5)
    remaining = max(0, user_calorie_targets["daily_total"] - consumed_so_far)
    return round(remaining)

def analyze_heart_rate_for_nutrition():
    """Analyze heart rate patterns to provide nutritional guidance."""
    # Get insights from yesterday's data and recent hours
    latest_hr_data = get_latest_hr_data(minutes=5)
    
    # Analyze current heart rate state
    current_hr = latest_hr_data['hr_mean'].mean()
    avg_hr = hourly_data['hr_mean'].mean()
    hr_state = "elevated" if current_hr > avg_hr * 1.1 else "normal" if current_hr > avg_hr * 0.9 else "low"
    
    # Check if user is in recovery period
    is_recovery = hourly_data['is_recovery'].mean() > 0.5
    
    # Check phase status
    is_out_of_phase = (latest_hr_data['phase_status'] == 'out-of-phase').mean() > 0.5
    
    return {
        "current_hr": current_hr,
        "hr_state": hr_state,
        "is_recovery": is_recovery,
        "is_out_of_phase": is_out_of_phase,
        "strain_index": hourly_data['strain_index'].mean()
    }

def recommend_food(query):
    """Generate food recommendations based on user query and heart rate data."""
    # Extract desired mood and meal time from query
    desired_mood = get_mood_from_phrase(query)
    meal_time = get_meal_time_from_phrase(query)
    
    # Analyze heart rate data for nutritional guidance
    hr_analysis = analyze_heart_rate_for_nutrition()
    
    # Get remaining calorie budget
    remaining_calories = get_remaining_calories()
    target_meal_calories = user_calorie_targets[meal_time] if meal_time != "snack" else user_calorie_targets["snacks"]
    adjusted_meal_calories = min(target_meal_calories, remaining_calories)
    
    # Consider last meal timing for recommendations
    hours_since_last_meal = (datetime.now() - user_last_meal["time"]).total_seconds() / 3600
    
    # Adjust recommendations based on heart rate analysis
    recommended_foods = []
    nutrition_advice = []
    
    # Modify recommendations based on heart rate state
    if hr_analysis["hr_state"] == "elevated":
        nutrition_advice.append("Your heart rate is elevated, suggesting foods that promote calmness and recovery.")
        # Blend in some calming foods with the desired mood foods
        recommended_foods = nutrition_database["calm"]["foods"][:2] + nutrition_database[desired_mood]["foods"][:3]
        nutrition_advice.append(f"Focus on these nutrients: {', '.join(nutrition_database['calm']['nutrients'][:3])}.")
    elif hr_analysis["is_recovery"]:
        nutrition_advice.append("Your body is in a recovery state. Focusing on restorative nutrition.")
        recommended_foods = nutrition_database["recovery"]["foods"][:2] + nutrition_database[desired_mood]["foods"][:3]
        nutrition_advice.append(f"Emphasize these nutrients: {', '.join(nutrition_database['recovery']['nutrients'][:3])}.")
    else:
        # Standard recommendation based on desired mood
        nutrition_advice.append(f"Based on your heart rate patterns and desire to feel {desired_mood}, here are some food suggestions.")
        recommended_foods = nutrition_database[desired_mood]["foods"][:5]
        nutrition_advice.append(f"These foods are rich in: {', '.join(nutrition_database[desired_mood]['nutrients'][:3])}.")
    
    # Adjust calorie recommendations based on heart rate strain
    if hr_analysis["strain_index"] > 7:
        nutrition_advice.append("Your heart has been under strain. Consider smaller, more frequent meals to maintain energy levels.")
        adjusted_meal_calories = min(adjusted_meal_calories, target_meal_calories * 0.8)  # Reduce meal size
    
    # Additional advice based on timing
    if hours_since_last_meal > 5:
        nutrition_advice.append("It's been over 5 hours since your last recorded meal. Consider having a balanced meal now to stabilize blood sugar.")
    elif hours_since_last_meal < 2 and meal_time != "snack":
        nutrition_advice.append("You had a meal recently. Consider a lighter option or waiting a bit longer before your next substantial meal.")
        
    # Find specific meal recommendations from our database that match the mood
    meal_recommendations = []
    for meal_name, meal_info in meal_database.items():
        if desired_mood in meal_info["mood"] and meal_info["calories"] <= adjusted_meal_calories * 1.2:
            meal_recommendations.append((meal_name, meal_info))
    
    # Sort by how close they are to the target calories
    meal_recommendations.sort(key=lambda x: abs(x[1]["calories"] - adjusted_meal_calories))
    
    # Prepare the response
    response = {
        "query": query,
        "desired_mood": desired_mood,
        "meal_time": meal_time,
        "heart_rate_analysis": hr_analysis,
        "remaining_calories": remaining_calories,
        "target_meal_calories": target_meal_calories,
        "adjusted_meal_calories": round(adjusted_meal_calories),
        "food_suggestions": recommended_foods,
        "meal_recommendations": meal_recommendations[:3],  # Top 3 meal recommendations
        "nutrition_advice": nutrition_advice,
        "foods_to_avoid": nutrition_database[desired_mood]["avoid"][:3]
    }
    
    return response

def display_food_recommendations(response):
    """Display food recommendations in a user-friendly format."""
    print(f"📋 PERSONALIZED NUTRITION RECOMMENDATIONS")
    print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print(f"🎯 Goal: Feel {response['desired_mood'].upper()} for {response['meal_time']}")
    print(f"💓 Current Heart Rate: {response['heart_rate_analysis']['current_hr']:.1f} BPM ({response['heart_rate_analysis']['hr_state']})")
    print(f"🔋 Calorie Budget: {response['remaining_calories']} calories remaining today")
    print(f"🍽️ Target for this {response['meal_time']}: {response['adjusted_meal_calories']} calories")
    print(f"\n💡 NUTRITION INSIGHTS:")
    for advice in response["nutrition_advice"]:
        print(f" • {advice}")
    
    print(f"\n🥗 RECOMMENDED FOODS:")
    for food in response["food_suggestions"]:
        print(f" • {food}")
    
    print(f"\n⚠️ FOODS TO LIMIT:")
    for food in response["foods_to_avoid"]:
        print(f" • {food}")
    
    print(f"\n🍳 MEAL IDEAS:")
    for meal_name, meal_info in response["meal_recommendations"]:
        print(f" • {meal_name.title()} ({meal_info['calories']} cal, {meal_info['protein']}g protein, {meal_info['carbs']}g carbs, {meal_info['fat']}g fat)")

# Test the recommendation system with a sample query
sample_query = "What type of food should I have to feel excited this afternoon?"
food_recommendations = recommend_food(sample_query)
display_food_recommendations(food_recommendations)

# Interactive query function
def ask_food_question():
    user_query = input("\nAsk a food recommendation question (e.g., 'What should I eat to stay focused tonight?'): ")
    if user_query.strip():
        recommendations = recommend_food(user_query)
        display_food_recommendations(recommendations)
    else:
        print("Please enter a valid question.")

# Uncomment to test with your own question
# ask_food_question()

print("\n✅ Nutrition recommendation system created!")

## 5. Creating a Previous Day's Data Based Personalization Function

Now we'll create a function that extracts insights from the previous day's heart rate data and generates personalized recommendations.

In [ ]:
def get_yesterday_insights():
    """Extract insights from yesterday's heart rate data to personalize recommendations."""
    # We're already using synthetic data for yesterday
    yesterday_data = hourly_data.copy()
    
    # Calculate key insights
    try:
        max_strain_index = yesterday_data['strain_index'].idxmax()
        max_strain_hour = yesterday_data.loc[max_strain_index, 'hour'].hour
        max_strain_value = yesterday_data.loc[max_strain_index, 'strain_index']
    except:
        # Fallback if there's an issue with the strain index
        max_strain_hour = 14  # Default to 2pm
        max_strain_value = 7.5
    
    # Get recovery hours (optimal times for low-intensity activities)
    recovery_hours = [h.hour for h in yesterday_data[yesterday_data['optimal_recovery'] == True]['hour']]
    if not recovery_hours:
        recovery_hours = [22, 23, 0, 1, 2, 3]  # Default to nighttime if none found
    
    # Get high heart rate periods
    high_hr_threshold = yesterday_data['hr_mean'].quantile(0.75)
    high_hr_periods = [h.hour for h in yesterday_data[yesterday_data['hr_mean'] > high_hr_threshold]['hour']]
    if not high_hr_periods:
        high_hr_periods = [12, 13, 14, 15]  # Default to afternoon if none found
    
    # Calculate current hour to make recommendations based on time of day
    current_hour = 8 #datetime.now().hour, in 24 hour format
    
    # Store insights in a dictionary
    insights = {
        "avg_heart_rate": yesterday_data['hr_mean'].mean(),
        "max_strain_hour": max_strain_hour,
        "max_strain_value": max_strain_value,
        "recovery_hours": recovery_hours,
        "high_hr_periods": high_hr_periods,
        "current_hour": current_hour
    }
    
    return insights

def generate_personalized_advice():
    """Generate personalized health advice based on heart rate patterns."""
    insights = get_yesterday_insights()
    
    # Generate time-appropriate advice
    advice = []
    current_hour = insights["current_hour"]
    
    # Morning advice (6am - 11am)
    if 6 <= current_hour <= 11:
        if insights["max_strain_value"] > 7:
            advice.append(f"Your heart experienced high strain yesterday around {insights['max_strain_hour']}:00. Consider a gentler morning routine today with light stretching or yoga.")
        
        if any(h in insights["high_hr_periods"] for h in range(current_hour-3, current_hour+1)):
            advice.append(f"Your heart rate tends to be higher around this time. Stay well hydrated this morning with at least 16oz of water.")
    
    # Afternoon advice (12pm - 5pm)
    elif 12 <= current_hour <= 17:
        if insights["avg_heart_rate"] > 80:
            advice.append("Your average heart rate was elevated yesterday. Consider a moderate-intensity workout today rather than high-intensity.")
        
        if current_hour in insights["recovery_hours"]:
            advice.append(f"Your body seems to recover well around this time ({current_hour}:00). This would be a good time for a short mindfulness break or deep breathing exercises.")
    
    # Evening advice (6pm - 10pm)
    elif 18 <= current_hour <= 22:
        if insights["max_strain_hour"] >= 18:
            advice.append("Your heart experienced peak strain in the evening yesterday. Consider shifting intense activities to earlier in the day.")
        
        recovery_evening_hours = [h for h in insights["recovery_hours"] if h >= 18]
        if recovery_evening_hours:
            recovery_times = ", ".join([f"{h}:00" for h in recovery_evening_hours])
            advice.append(f"Your optimal recovery periods in the evening are around {recovery_times}. These are great times for gentle stretching or relaxation.")
    
    # Night advice (11pm - 5am)
    else:
        advice.append("Your heart rate data suggests you should be resting now. Focus on quality sleep to optimize recovery.")
    
    # General advice based on yesterday's patterns
    if insights["max_strain_value"] > 8:
        advice.append(f"Yesterday had periods of high heart strain. Prioritize recovery today with extra hydration and lower-intensity activities.")
    
    # Add hydration advice based on high HR periods
    high_hr_str = ", ".join([f"{h}:00" for h in insights["high_hr_periods"]])
    advice.append(f"Your heart rate was elevated at {high_hr_str} yesterday. Ensure you drink more water during these times today.")
    
    # If no specific advice was generated, add a default message
    if not advice:
        advice.append("Based on your heart rate patterns, maintain your regular routine with adequate hydration and rest periods.")
    
    return "\n\n".join(advice)

# Test the personalization function
personalized_advice = generate_personalized_advice()
print("Personalized Advice Based on Heart Rate:\n")
print(personalized_advice)

print("\n✅ Personalization function created!")

## 6. Building a Personalized Multi-Agent RAG System

Now let's create a personalized RAG system that leverages heart rate insights in its recommendations.

In [ ]:
# Create a personalized retrieval function that combines heart rate insights and health tips
def retrieve_personalized_tips(query: str) -> str:
    """Retrieve health tips and personalize them based on heart rate data."""
    # Get standard tips
    standard_tips = retrieve_tips(query)
    
    # Get personalized advice based on yesterday's data
    personalized_advice = generate_personalized_advice()
    
    # Combine both
    return f"PERSONALIZED ADVICE BASED ON YOUR HEART RATE DATA:\n{personalized_advice}\n\nGENERAL HEALTH TIPS:\n{standard_tips}"

# For testing without Azure AI Foundry, create a mock response for the agents
class MockResponseGenerator:
    def __init__(self, name):
        self.name = name
    
    def generate_response(self, message):
        if self.name == "PersonalizedRetrieverAgent":
            # This agent would fetch the data
            return f"I've retrieved the personalized health information based on your heart rate data."
        else:
            # This agent would craft a response
            insights = get_yesterday_insights()
            recovery_times = ", ".join([f"{h}:00" for h in insights["recovery_hours"][:3]])
            return (f"Based on your heart rate data, I recommend scheduling activities around your natural recovery periods at {recovery_times}. "
                   f"Your heart experienced the most strain yesterday around {insights['max_strain_hour']}:00, so today would be good for gentler activities."
                   f"Remember to hydrate well, especially around {insights['high_hr_periods'][0]}:00 when your heart rate tends to be elevated.")

# Create the personalized agents
try:
    # Create a personalized version of our agent team
    personalized_retriever = AssistantAgent(
        name="PersonalizedRetrieverAgent",
        system_message=(
            "You are a smart health data analyzer and retrieval agent. "
            "Your task is to fetch relevant fitness and health tips based on the user's query, "
            "and personalize them based on their heart rate data from yesterday. "
            "Use the provided tool 'retrieve_personalized_tips' to get the information."
        ),
        model_client=model_client,
        tools=[retrieve_personalized_tips],
        reflect_on_tool_use=True
    )

    personalized_responder = AssistantAgent(
        name="PersonalizedResponderAgent",
        system_message=(
            "You are a friendly and personalized fitness coach. "
            "Use both the general health tips and personalized heart rate insights to craft "
            "a response that is specifically tailored to the user's current physiological state. "
            "Reference their heart rate patterns and recovery needs when providing advice."
        ),
        model_client=model_client
    )

    personalized_group_chat = RoundRobinGroupChat(
        agents=[personalized_retriever, personalized_responder],
        termination_condition=None,
        max_turns=4
    )
    
    print("✅ Personalized RAG system created with Azure AI Foundry!")
    using_mock = False
    
except Exception as e:
    print(f"⚠️ Could not create agents with Azure AI Foundry: {e}")
    print("⚠️ Using mock agents for demonstration purposes...")
    
    # Create mock agents
    retriever_mock = MockResponseGenerator("PersonalizedRetrieverAgent")
    responder_mock = MockResponseGenerator("PersonalizedResponderAgent")
    using_mock = True
    
    print("✅ Mock personalized RAG system created for demonstration!")